### Links
- https://leojosefm.medium.com/python-analyzing-itunes-library-97bec60e13cb
- https://pitchfork.com/features/lists-and-guides/25-years-of-pitchfork-reviews-by-the-numbers/
- https://www.kaggle.com/ermoore/pitchfork-reviews-through-12617#
- https://ucladatares.medium.com/visualizing-trends-and-patterns-in-pitchfork-reviews-cad1a130869b

In [1]:
# It's possible to get playlist data from lib

In [2]:
import pandas as pd
import numpy as np
import xml.etree.ElementTree as ET # XML parsing
import matplotlib.pyplot as plt

from datetime import date
from wordcloud import WordCloud

In [27]:
def round_down(num, divisor):
    return num - (num%divisor)

def cols(kind):
    cols = []
    for i in range(len(kind)):
        for j in range(len(kind[i])):
            if kind[i][j].tag == "key":
                cols.append(kind[i][j].text)
    return list(set(cols))

def df_creation(kind,cols):
    df = pd.DataFrame(columns = cols)
    dict1 = {}
    for i in range(len(kind)):
        for j in range(len(kind[i])):
            if kind[i][j].tag == "key":
                dict1[kind[i][j].text] = kind[i][j+1].text
        list_values = [i for i in dict1.values()]
        list_keys = [j for j in dict1.keys()]
        dict1 = {}
        df_temp = pd.DataFrame([list_values],columns = list_keys)
        df = pd.concat([df,df_temp],axis = 0,ignore_index = True,sort = True)
    return df

In [4]:
# Load Data
reviews_raw = pd.read_csv("/Users/kylezengo/Desktop/DS/Music Ratings/reviews.csv")

top1000_raw = open("/Users/kylezengo/Desktop/DS/Music Ratings/1-1000.txt")

lib = "/Users/kylezengo/Desktop/DS/Music Ratings/Library_20241017.xml"

In [5]:
# Clean up
reviews = reviews_raw.replace({'artist': "Run the Jewels"}, "Run The Jewels")
reviews = reviews.replace({'artist': "Gojira"}, "GOJIRA")
reviews = reviews.replace({'artist': "Nicolás Jaar"}, "Nicolas Jaar")
reviews = reviews.replace({'album': "Keep it Like a Secret"}, "Keep It Like a Secret")

reviews.columns = "pitchfork_"+ reviews.columns
reviews = reviews.rename(columns={"pitchfork_artist": "artist"})
reviews = reviews.rename(columns={"pitchfork_album": "album"})

top1000 = top1000_raw.read().splitlines()

In [6]:
tree = ET.parse(lib)
root = tree.getroot()
main_dict = root.findall('dict')

tracks_dict = None
playlist_dict = None
for item in list(main_dict[0]):
    if item.tag == "dict" and tracks_dict is None:
        tracks_dict = item  # Save the dict for tracks
    elif item.tag == "array" and playlist_dict is None:
        playlist_dict = item  # Save the array for playlists
    
    # Break if both have been found
    if tracks_dict is not None and playlist_dict is not None:
        break
        
tracklist = list(tracks_dict.findall('dict'))
playlists = playlist_dict.findall('dict')

In [7]:
# Loop through the playlists to find the one with Name "Party"
track_ids = []
for playlist in playlists:
    items = list(playlist)
    for j in range(len(items)):
        if items[j].tag == 'key' and items[j].text == 'Name' and items[j + 1].text == 'Party':
            # Found the "Party" playlist, now find its "Playlist Items"
            for k in range(len(items)):
                if items[k].tag == 'key' and items[k].text == 'Playlist Items':
                    # Find the array containing the track IDs
                    playlist_items = items[k + 1]
                    if playlist_items.tag == 'array':
                        for track_item in playlist_items.findall('dict'):
                            # Look for the "Track ID" inside each <dict>
                            for track_id in track_item.findall('integer'):
                                track_ids.append(int(track_id.text))

In [8]:
podcast = []     # All podcast elements
purchased = []   # All purchased music
apple_music = [] # Music added to lirary through subscription
apple_lossless_audio_file = [] 
AIFF_audio_file = []
MPEG_audio_file = []
AAC_audio_file = []
protected_AAC_audio_file = []
WAV_audio_file = []

for item in tracklist:
    #x=item.getchildren()
    x = list(item)
    for i in range(len(x)):
        if x[i].text == "Genre" and x[i+1].text == "Podcast": #
            #podcast.append(item.getchildren())
            podcast.append(list(item))
        if x[i].text == "Kind" and x[i+1].text == "Purchased AAC audio file":
            #purchased.append(item.getchildren())
            purchased.append(list(item)) 
        if x[i].text == "Kind" and x[i+1].text == "Apple Music AAC audio file":
            #apple_music.append(item.getchildren())
            apple_music.append(list(item))
        if x[i].text == "Kind" and x[i+1].text == "Apple Lossless audio file":
            apple_lossless_audio_file.append(list(item))
        if x[i].text == "Kind" and x[i+1].text == "AIFF audio file":
            AIFF_audio_file.append(list(item))     
        if x[i].text == "Kind" and x[i+1].text == "MPEG audio file":
            MPEG_audio_file.append(list(item))     
        if x[i].text == "Kind" and x[i+1].text == "AAC audio file":
            AAC_audio_file.append(list(item))     
        if x[i].text == "Kind" and x[i+1].text == "Protected AAC audio file":
            protected_AAC_audio_file.append(list(item))                 
        if x[i].text == "Kind" and x[i+1].text == "WAV audio file":
            WAV_audio_file.append(list(item))                             

In [9]:
print ("Number of tracks under Podcast: ",len(podcast))
print ("Number of tracks Purchased: ",len(purchased))
print ("Number of Music added thought Apple Music subscription: ",len(apple_music))
print ("Number of tracks Apple Lossless audio file: ",len(apple_lossless_audio_file))
print ("Number of tracks AIFF audio file: ",len(AIFF_audio_file))
print ("Number of tracks MPEG audio file: ",len(MPEG_audio_file))
print ("Number of tracks AAC audio file: ",len(AAC_audio_file))
print ("Number of tracks Protected AAC audio file: ",len(protected_AAC_audio_file))
print ("Number of tracks WAV audio file: ",len(WAV_audio_file))
print ("Total tracks: ",len(podcast) + len(purchased) + len(apple_music) + len(apple_lossless_audio_file)
      + len(AIFF_audio_file) + len(MPEG_audio_file) + len(AAC_audio_file) + len(protected_AAC_audio_file)
      + len(WAV_audio_file))

Number of tracks under Podcast:  0
Number of tracks Purchased:  4080
Number of Music added thought Apple Music subscription:  0
Number of tracks Apple Lossless audio file:  354
Number of tracks AIFF audio file:  39
Number of tracks MPEG audio file:  5023
Number of tracks AAC audio file:  133
Number of tracks Protected AAC audio file:  188
Number of tracks WAV audio file:  1
Total tracks:  9818


In [10]:
df_podcast = df_creation(podcast,cols(podcast))

In [11]:
df_purchased = df_creation(purchased,cols(purchased))

In [12]:
df_apple_music = df_creation(apple_music,cols(apple_music))

In [13]:
df_apple_lossless_audio_file = df_creation(apple_lossless_audio_file, cols(apple_lossless_audio_file))

In [14]:
df_AIFF_audio_file = df_creation(AIFF_audio_file, cols(AIFF_audio_file))

In [15]:
df_MPEG_audio_file = df_creation(MPEG_audio_file, cols(MPEG_audio_file))

In [16]:
df_AAC_audio_file = df_creation(AAC_audio_file, cols(AAC_audio_file))

In [17]:
df_protected_AAC_audio_file = df_creation(protected_AAC_audio_file, cols(protected_AAC_audio_file))

In [18]:
df_WAV_audio_file = df_creation(WAV_audio_file, cols(WAV_audio_file))

In [19]:
# Total Time in ms (divide by 1,000 to convert to seconds)
columns_of_interest = ['track_id','song_name','play_count','skip_count','album','artist'
                     ,'genre',"kind",'persistent_id','year_of_release','play_date','skip_date'
                     ,'release_date','date_modified','date_added','disc_count','disc_number'
                     ,'track_count','track_number','total_time']

temp_purchased = df_purchased.loc[:,['Track ID','Name','Play Count','Skip Count'
                     ,'Album','Artist','Genre','Kind','Persistent ID','Year'
                     ,'Play Date UTC','Skip Date','Release Date','Date Modified','Date Added'
                     ,'Disc Count','Disc Number','Track Count','Track Number','Total Time']]

temp_apple_lossless_audio_file = df_apple_lossless_audio_file.loc[:,["Track ID","Name","Play Count","Skip Count"
                     ,"Album","Artist","Genre","Kind","Persistent ID","Year"
                     ,"Play Date UTC","Skip Date","Date Modified",'Date Added'
                     ,'Disc Count','Disc Number','Track Count','Track Number','Total Time']]
                     # Dropped 'Release Date'

temp_AIFF_audio_file = df_AIFF_audio_file.loc[:,["Track ID","Name","Play Count","Skip Count","Album"
                     ,"Artist","Genre","Kind","Persistent ID","Year"
                     ,"Play Date UTC","Skip Date","Date Modified",'Date Added'
                     ,'Disc Count','Disc Number','Track Count','Track Number','Total Time']]
                     # Dropped 'Release Date'

temp_MPEG_audio_file = df_MPEG_audio_file.loc[:,["Track ID","Name","Play Count","Skip Count","Album"
                     ,"Artist","Genre","Kind","Persistent ID","Year"
                     ,"Play Date UTC","Skip Date","Release Date","Date Modified",'Date Added'
                     ,'Disc Count','Disc Number','Track Count','Track Number','Total Time']]

temp_AAC_audio_file = df_AAC_audio_file.loc[:,["Track ID","Name","Play Count","Skip Count","Album"
                     ,"Artist","Genre","Kind","Persistent ID","Year"
                     ,"Play Date UTC","Skip Date","Release Date","Date Modified",'Date Added'
                     ,'Disc Count','Disc Number','Track Count','Track Number','Total Time']]

temp_protected_AAC_audio_file = df_protected_AAC_audio_file.loc[:,["Track ID","Name","Play Count","Skip Count"
                     ,"Album","Artist","Genre","Kind","Persistent ID","Year"
                     ,"Play Date UTC","Skip Date","Release Date","Date Modified",'Date Added'
                     ,'Disc Count','Disc Number','Track Count','Track Number','Total Time']]

temp_WAV_audio_file = df_WAV_audio_file.loc[:,["Track ID","Name","Play Count","Skip Count"
                     ,"Kind","Persistent ID","Play Date UTC","Skip Date"
                     ,"Date Modified",'Date Added','Total Time']]
                     # Dropped 'Release Date','Album','Artist','Genre','Year','Disc Count'
                     # ,'Disc Number','Track Count','Track Number'

In [23]:
df_songs = pd.concat([temp_purchased,temp_apple_lossless_audio_file,temp_AIFF_audio_file
                     ,temp_MPEG_audio_file,temp_AAC_audio_file,temp_protected_AAC_audio_file
                      ,temp_WAV_audio_file],axis = 0)
df_songs.columns = columns_of_interest

df_songs[['track_id','play_count','skip_count','year_of_release','disc_count','disc_number','track_count'
          ,'track_number','total_time']]  = df_songs[['track_id','play_count','skip_count','year_of_release'
                                                      ,'disc_count','disc_number','track_count','track_number'
                                                      ,'total_time']].apply(pd.to_numeric)

df_songs[['play_date','skip_date','release_date','date_modified']] = df_songs[['play_date',
        'skip_date','release_date','date_modified']].apply(pd.to_datetime)

df_songs['total_listen_time'] = df_songs['play_count'] * df_songs['total_time']
df_songs['playMinusSkip'] = df_songs['play_count'] - df_songs['skip_count']
df_songs['playMinusSkip_time'] = df_songs['playMinusSkip'] * df_songs['total_time']

df_songs['year_of_release'] = df_songs['year_of_release'].fillna(0).astype(int)

df_songs['decade'] = round_down(df_songs['year_of_release'],10).astype(int)

# Genre clean up
df_songs['genre_clean'] = df_songs['genre']
df_songs.loc[df_songs['genre']=="www.CamelbackMusic.com",'genre_clean'] = None
df_songs.loc[df_songs['genre']=="www.mp3-ogg.ru",'genre_clean'] = None
df_songs.loc[df_songs['genre']=="data",'genre_clean'] = None
df_songs.loc[df_songs['genre']=="Other",'genre_clean'] = None
df_songs.loc[df_songs['genre']=="Unknown genre",'genre_clean'] = None
df_songs.loc[df_songs['genre']==" ",'genre_clean'] = None
df_songs.loc[df_songs['genre']=="131",'genre_clean'] = None
df_songs.loc[df_songs['genre']=="144",'genre_clean'] = None
df_songs.loc[df_songs['genre']=="60 S",'genre_clean'] = None
df_songs.loc[df_songs['genre']=="00s",'genre_clean'] = None

df_songs.loc[df_songs['genre']=="Alt. Rock",'genre_clean'] = "Alternative Rock"

df_songs.loc[df_songs['genre']=="Electro-House",'genre_clean'] = "Electro House"

df_songs.loc[df_songs['genre']=="Electonic",'genre_clean'] = "Electronic"

df_songs.loc[df_songs['genre']=="Hip Hop",'genre_clean'] = "Hip-Hop"
df_songs.loc[df_songs['genre']=="Hip - Hop",'genre_clean'] = "Hip-Hop"


df_songs.loc[df_songs['genre']=="[Hip-Hop/Rap]",'genre_clean'] = "Hip-Hop/Rap"
df_songs.loc[df_songs['genre']=="Hip Hop/Rap",'genre_clean'] = "Hip-Hop/Rap"
df_songs.loc[df_songs['genre']=="Rap & Hip-Hop",'genre_clean'] = "Hip-Hop/Rap"

df_songs.loc[df_songs['genre']=="Post-Rock",'genre_clean'] = "Post Rock"

df_songs.loc[df_songs['genre']=="Other Reggae",'genre_clean'] = "Reggae"

df_songs.loc[df_songs['genre']=="Synth-pop",'genre_clean'] = "Synthpop"

# Add on pitchfork ratings
df_songs = df_songs.merge(reviews, how='left', on=['album','artist'])

# Playlists
df_songs['Party'] = df_songs['track_id'].isin(track_ids)

df_songs.dtypes

track_id                            int64
song_name                          object
play_count                        float64
skip_count                        float64
album                              object
artist                             object
genre                              object
kind                               object
persistent_id                      object
year_of_release                     int64
play_date             datetime64[ns, UTC]
skip_date             datetime64[ns, UTC]
release_date          datetime64[ns, UTC]
date_modified         datetime64[ns, UTC]
date_added                         object
disc_count                        float64
disc_number                       float64
track_count                       float64
track_number                      float64
total_time                          int64
total_listen_time                 float64
playMinusSkip                     float64
playMinusSkip_time                float64
decade                            

In [26]:
# For a playlist - aiming at a genre/ vibe
# which songs?
# highly rated / played by me
party = df_songs
party = party.loc[party['genre_clean']!="Classical"]
party = party.loc[party['genre_clean']!="Ambient"]
party = party.loc[party['genre_clean']!="Dubstep"]
party = party.loc[party['genre_clean']!="Instrumental Rock"]
party = party.loc[party['genre_clean']!="Post Rock"]
party = party.loc[party['genre_clean']!="Metal"]

party = party.loc[party['pitchfork_genre']!="Metal"]

party = party.loc[party['artist']!="Opiuo"]
party = party.loc[party['artist']!="The Sword"]
party = party.loc[party['artist']!="Metallica"]
party = party.loc[party['artist']!="50 Cent"]
party = party.loc[party['artist']!="Death Grips"]
party = party.loc[party['artist']!="Phoenix"]

party = party.loc[party['song_name']!="Her (Flim Score)"]
party = party.loc[party['song_name']!="Moonchild"]
party = party.loc[party['song_name']!="Skinny Love"]
party = party.loc[party['song_name']!="Hate Or Glory"]

# party = party.loc[party['pitchfork_score']>=7]
party = party.loc[party['Party']==True]
party = party.sort_values(by='playMinusSkip', ascending=False).reset_index(drop=True)
party = party.groupby('artist').first()
party = party.sort_values(by='playMinusSkip', ascending=False).reset_index()

print(f'{party.shape[0]} songs')
party[['song_name','artist','album','genre_clean','year_of_release','pitchfork_genre'
       ,'pitchfork_score']].head(30)

66 songs


,song_name,artist,album,genre_clean,year_of_release,pitchfork_genre,pitchfork_score
0,Obvs,Jamie xx,In Colour,Electronic,2015,Electronic,9.3
1,"Do It, Try It",M83,Junk,Electronic,2016,Electronic,6.8
2,Great DJ,The Ting Tings,We Started Nothing,None,2008,Electronic,3.8
3,On The Regular,Shamir,Ratchet,Electronic,2015,Pop/R&B,8.3
4,Change Your Life (You Can Do It),Dan Deacon,Change Your Life (You Can Do It) - Single,Electronic,2016,None,NaN
5,New Dorp. New York (feat. Ezra Koenig),SBTRKT,Wonder Where We Land,Electronic,2014,Electronic,5.5
6,Innerbloom,RÜFÜS DU SOL,Bloom,Electronic,2015,None,NaN
7,Kyle (I Found You),Fred again..,Actual Life (April 14 - December 17 2020),Electronic,2019,None,NaN
8,Marea (We’ve Lost Dancing),Fred again.. & The Blessed Madonna,Actual Life (April 14 - December 17 2020),Electronic,2021,None,NaN
9,Punk Is Dead,Jeffrey Lewis,12 Crass Songs,Alternative,2007,None,NaN


In [ ]:
reviews.loc[reviews['album']=="Singles"]

In [ ]:
# Artists
df_artist = df_songs.groupby(['artist'])[['play_count','total_listen_time']].sum().reset_index()

df_artist.sort_values(by='total_listen_time', ascending=False).reset_index(drop=True).head(10)

In [ ]:
# Decades
df_decade = df_songs.groupby(['decade']).agg({'play_count': 'sum'
                                             ,'total_listen_time': 'sum'
                                             ,'song_name': 'count'}).reset_index()

plt.bar(df_decade['decade'], df_decade['song_name'], width = 8)
plt.show()

In [ ]:
# Genres
df_genre = df_songs.groupby(['genre_clean']).agg({'play_count': 'sum'
                                             ,'total_listen_time': 'sum'
                                             ,'song_name': 'count'}).reset_index()
df_genre = df_genre.rename(columns={"song_name": "song_count"})

df_genre['total_listen_time_hms'] = pd.to_timedelta(df_genre['total_listen_time'], unit="ms")

df_genre.sort_values(by='total_listen_time', ascending=False).reset_index(drop=True).head(10)

In [ ]:
df_songs['disc_number'] = df_songs['disc_number'].fillna(1)

# groupby dropps NaN's we replace NaN with 1 about, so this behavior is okay.
# Can add dropna = False to groupby if need to review
albumSongCount = df_songs.groupby(['album','artist','disc_number']).agg({'track_count': 'max'}).reset_index()
albumSongCount = albumSongCount.groupby(['album','artist']).agg({'track_count': 'sum'}).reset_index()

In [ ]:
df_albums = df_songs.groupby(['album','artist']).agg({'play_count': 'sum'
                                                      ,'skip_count': 'sum'
                                                      ,'total_listen_time': 'sum'
                                                      ,'playMinusSkip': 'sum'
                                                      ,'playMinusSkip_time': 'sum'
                                                      ,'song_name': 'count'
                                                      ,'disc_count': 'max'
                                                      ,'date_added': 'min'
                                                      ,'year_of_release': 'min'}).reset_index()

df_albums = df_albums.merge(albumSongCount, how='left', on=['album','artist'])

df_albums['date_added'] = df_albums['date_added'].apply(pd.to_datetime)

df_albums['lib_time'] = pd.to_datetime(date.today()) - pd.DatetimeIndex(df_albums['date_added']).tz_localize(None)
df_albums['lib_time'] = df_albums['lib_time'].dt.days

df_albums = df_albums.rename(columns={"song_name": "song_count_lib"})

# Will get warning if run same day as adding new songs to library due to ln(0). Will resolve next day!
df_albums['PC_ratio'] = df_albums['play_count'] / df_albums['lib_time']
df_albums['PCT_ratio'] = df_albums['total_listen_time'] / df_albums['lib_time']
df_albums['SC_ratio'] = df_albums['skip_count'] / df_albums['lib_time']
df_albums['PMS_ratio'] = df_albums['playMinusSkip'] / df_albums['lib_time']
df_albums['PMST_ratio'] = df_albums['playMinusSkip_time'] / df_albums['lib_time']
df_albums['PS_ratio'] = df_albums['play_count'] / df_albums['skip_count']
df_albums['album_complete'] = df_albums['song_count_lib'] / df_albums['track_count']
df_albums['log_lib_time'] = np.log(df_albums['lib_time'])
df_albums['PCT_ratio'] = df_albums['total_listen_time'] / df_albums['lib_time']
df_albums['PCT_L_ratio'] = df_albums['total_listen_time'] / df_albums['log_lib_time']
df_albums['for_rating'] = df_albums['PS_ratio'] * df_albums['album_complete'] * df_albums['PCT_ratio']
df_albums['for_rating_L'] = df_albums['PS_ratio'] * df_albums['album_complete'] * df_albums['PCT_L_ratio']

df_albums.replace([np.inf, -np.inf], np.nan, inplace=True)

df_albums = df_albums.merge(reviews, how='left', on=['album','artist'])

df_albums.sort_values(by='total_listen_time', ascending=False).head()

In [ ]:
print(df_albums.shape)
print(df_albums.loc[df_albums['pitchfork_score'].isna()].shape)
print(df_albums.loc[df_albums['pitchfork_score'].isna()].shape[0]/df_albums.shape[0])

In [ ]:
album10 = df_albums.loc[df_albums['score']==10]
album10.sort_values(by='total_listen_time', ascending=False).reset_index(drop=True)

In [ ]:
# Filter to only albums added over one year ago
d = date.today().replace(year = date.today().year - 1)
d = pd.to_datetime(d).tz_localize(tz = 'UTC')

df_albums_1yr = df_albums[df_albums['date_added'] < d]
# df_albums_1yr = df_albums[df_albums['date_added'] < d0]
df_albums_1yr = df_albums_1yr[df_albums_1yr['album_complete'] <= 1]
df_albums_1yr = df_albums_1yr.sort_values(by='for_rating', ascending=False)

rating = []

for i in range(len(df_albums_1yr)+1):
    for j in range(101):
        if i > j*len(df_albums_1yr)/101 and i <= (j+1)*len(df_albums_1yr)/101 :
            rating.append(100-j)
        
df_albums_1yr['rating'] = rating
df_albums_1yr['rating'] = df_albums_1yr['rating']/10

In [ ]:
# Hist rating
df_albums_1yr.hist('rating')

plt.show()

In [ ]:
df_albums_1yr.shape

In [ ]:
mu, sigma = 5, 1.5 # mean and standard deviation
s = np.random.normal(mu, sigma, len(df_albums_1yr))

s = np.sort(s)
s = s[::-1]

df_albums_1yr = df_albums_1yr.sort_values(by='for_rating', ascending=False)

df_albums_1yr['norm'] = s
df_albums_1yr['norm'] = df_albums_1yr['norm'].round(1)

df_albums_1yr.hist('norm',bins=50)

plt.show()

In [ ]:
# pd.set_option("max_columns", None)

df_albums_1yr['decade'] = round_down(df_albums_1yr['year_of_release'],10)

df_albums_1yr_1950 = df_albums_1yr[df_albums_1yr['decade'] == 1950]
df_albums_1yr_1960 = df_albums_1yr[df_albums_1yr['decade'] == 1960]
df_albums_1yr_1970 = df_albums_1yr[df_albums_1yr['decade'] == 1970]
df_albums_1yr_1980 = df_albums_1yr[df_albums_1yr['decade'] == 1980]
df_albums_1yr_1990 = df_albums_1yr[df_albums_1yr['decade'] == 1990]
df_albums_1yr_2000 = df_albums_1yr[df_albums_1yr['decade'] == 2000]
df_albums_1yr_2010 = df_albums_1yr[df_albums_1yr['decade'] == 2010]


df_albums_1yr.sort_values(by = 'for_rating', ascending = False).head(20)
# df_albums_1yr_1950.sort_values(by = 'for_rating', ascending = False).head()
# df_albums_1yr_1960.sort_values(by = 'for_rating', ascending = False).head()
# df_albums_1yr_1970.sort_values(by = 'for_rating', ascending = False).head()
# df_albums_1yr_1980.sort_values(by = 'for_rating', ascending = False).head()
# df_albums_1yr_1990.sort_values(by = 'for_rating', ascending = False).head()
# df_albums_1yr_2000.sort_values(by = 'for_rating', ascending = False).head()
# df_albums_1yr_2010.sort_values(by = 'for_rating', ascending = False).head()

In [ ]:
# Check cases where have more songs in album than total track number
TEST = df_albums[df_albums['album_complete'] > 1]
TEST.sort_values(by='album_complete', ascending=False)

In [ ]:
reviews['review'].fillna('', inplace=True)

# Create string of all text in all reviews
text = " ".join(item for item in reviews['review'])

# Build word cloud
wordcloud = WordCloud(background_color = "white", collocations = False, min_word_length = 2).generate(text)

plt.figure(figsize = (8, 8), facecolor = None) 
plt.imshow(wordcloud) 
plt.axis("off") 
plt.tight_layout(pad = 0) 
plt.show()